In [ ]:
import os, sys
project_root_dir = os.path.join(os.getcwd(),'../..')
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)

from matplotlib import pyplot as plt
import numpy as np
import torch
import config

from utils import show_abundance, plot_endmembers
from dataset import SyntheticDataset, SyntheticDatasetType
from HySpecLab.metrics import rmse, sad

In [ ]:
from dataset import JasperRidge
dataset = JasperRidge(config.JasperRidge_PATH)
dataset_name = 'JasperRidge'

result_path = os.path.join(config.RESULTS_PATH, 'jasperRidge/matlab')

In [ ]:
from scipy import io as sio

endmember_estimation = sio.loadmat(os.path.join(result_path, 'endmember_estimation.mat'))
vca_ee = torch.tensor(endmember_estimation['VCA'])
nfindr_ee = torch.tensor(endmember_estimation['NFINDR'])

sad_result = sad(vca_ee, dataset.endmembers())
vca_idx = torch.argmin(sad_result, dim=0) # Index for reordering the ground truth

sad_result = sad(nfindr_ee, dataset.endmembers())
nfindr_idx = torch.argmin(sad_result, dim=0) # Index for reordering the ground truth

vca_ee = vca_ee[vca_idx]
nfindr_ee = nfindr_ee[nfindr_idx]

In [ ]:
fig = plot_endmembers(dataset.endmembers(), ticks_range=(0, 1), 
                      endmember_estimation=[nfindr_ee, vca_ee],
                      ee_labels=['Ground Truth', 'NFINDR', 'VCA'])
plt.show(fig)
# fig.savefig(os.path.join(result_path, 'imgs/endmembers_estimation.pdf'), bbox_inches='tight')

In [ ]:
from HySpecLab.metrics import sad

sad_result_vca = sad(vca_ee, dataset.endmembers())
print(sad_result_vca, np.diagonal(sad_result_vca.numpy()).mean())
sad_vca = np.diagonal(sad_result_vca.numpy()).mean()

sad_result_nfindr = sad(nfindr_ee, dataset.endmembers())
print(sad_result_nfindr, np.diagonal(sad_result_nfindr.numpy()).mean())
sad_nfindr = np.diagonal(sad_result_nfindr.numpy()).mean()

In [ ]:
FCLS = sio.loadmat(os.path.join(result_path, 'FCLS.mat'))
save_dir = os.path.join(result_path, 'imgs/LMM')

X_true = dataset.A @ dataset.endmembers()


Xhat = FCLS['Xhat_NFINDR'].T

nfindr_reconstruct = torch.tensor(FCLS['Xhat_NFINDR'].T)
vca_reconstruct = torch.tensor(FCLS['Xhat_VCA'].T)
nfindr_A = torch.tensor(FCLS['A_NFINDR'].T)
vca_A = torch.tensor(FCLS['A_VCA'].T)
# dataframe which contains the RMSE
# Saving metrics
import pandas as pd
lmm_df = pd.DataFrame(columns=['Method', 'RMSE_X', 'RMSE_A', 'SAD_M'])
lmm_df['Method'] = ['LMM+NFINDR', 'LMM+VCA']
lmm_df['RMSE_X'] = [rmse(X_true, nfindr_reconstruct, dim=None).numpy(), rmse(X_true, vca_reconstruct, dim=None).numpy()]
lmm_df['RMSE_A'] = [rmse(dataset.A, nfindr_A, dim=None).numpy(), rmse(dataset.A, vca_A, dim=None).numpy()]
lmm_df['SAD_M'] = [sad_nfindr, sad_vca]
lmm_df

In [ ]:
dataset.n_col

In [ ]:
A = FCLS['A_NFINDR'].T.reshape(dataset.n_row,dataset.n_col,-1,order='C')
fig = show_abundance(A[:, :, nfindr_idx])
plt.show(fig)
fig.savefig(os.path.join(save_dir, 'A_NFINDR.png'), dpi=300, bbox_inches='tight')

A = FCLS['A_VCA'].T.reshape(dataset.n_row,dataset.n_col,-1,order='C')
fig = show_abundance(A[:, :, vca_idx])
plt.show(fig)
fig.savefig(os.path.join(save_dir, 'A_VCA.png'), dpi=300, bbox_inches='tight')

# ELMM

In [ ]:
ELMM = sio.loadmat(os.path.join(result_path, 'ELMM.mat'))
save_dir = os.path.join(result_path, 'imgs/ELMM')

from HySpecLab.metrics import rmse
nfindr_reconstruct = torch.tensor(ELMM['Xhat_NFINDR'].T)
vca_reconstruct = torch.tensor(ELMM['Xhat_VCA'].T)
nfindr_A = torch.tensor(ELMM['A_NFINDR'].T)
vca_A = torch.tensor(ELMM['A_VCA'].T)

import pandas as pd
elmm_df = pd.DataFrame(columns=['Method', 'RMSE_X', 'RMSE_A', 'SAD_M'])
elmm_df['Method'] = ['ELMM+NFINDR', 'ELMM+VCA']
elmm_df['RMSE_X'] = [rmse(X_true, nfindr_reconstruct, dim=None).numpy(), rmse(X_true, vca_reconstruct, dim=None).numpy()]
elmm_df['RMSE_A'] = [rmse(dataset.A, nfindr_A, dim=None).numpy(), rmse(dataset.A, vca_A, dim=None).numpy()]
elmm_df['SAD_M'] = [sad_nfindr, sad_vca]

elmm_df

In [ ]:
from utils import show_abundance
A = ELMM['A_NFINDR'].T.reshape(dataset.n_row, dataset.n_col, -1,order='C')
fig = show_abundance(np.abs(A[:, :, nfindr_idx]))
plt.show(fig)
fig.savefig(os.path.join(save_dir, 'A_NFINDR.png'), dpi=300, bbox_inches='tight')

A = ELMM['A_VCA'].T.reshape(dataset.n_row, dataset.n_col,-1,order='C')
fig = show_abundance(np.abs(A[:, :, vca_idx]))
plt.show(fig)
fig.savefig(os.path.join(save_dir, 'A_VCA.png'), dpi=300, bbox_inches='tight')

# Save Dataframe

In [ ]:
df = pd.concat([lmm_df, elmm_df])
df.to_csv(os.path.join(result_path, 'metrics.csv'), index=False)
df